In [ ]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold, cross_val_score, cross_val_predict
from scipy.interpolate import griddata
import statsmodels.api as sm

In [ ]:
# Load data from the .mat file
data = sio.loadmat('../data/ML_Optimal_method.mat')
eddy_double_LES = data['eddy_double_LES']
eddy_double_TBNN = data['eddy_double_TBNN']
Sij = data['Sij']
Rij = data['Rij']
x = data['x']
y = data['y']
x_point = data['x_point']
y_point = data['y_point']
bij_LES = data['bij_LES']
bij_TBNN = data['bij_TBNN']

### edd viscosity = function(Sij, Rij)

In [ ]:
# Prepare features and target variables
# Use columns 0, 1, 3, and 4 from Sij an Rij as features
X = np.concatenate((Sij[:, [0, 1, 3, 4]], Rij[:, [0, 1, 3, 4]]), axis=1)
Y = eddy_double_TBNN.ravel()
Y_LES = eddy_double_LES.ravel()

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Use PolynomialFeatures to create higher order terms
# degree = 2 indicates quadratic terms (set degree to 2, 3, 4, etc. as needed)
poly = PolynomialFeatures(degree=4, include_bias=True)
X_poly = poly.fit_transform(X_scaled)

# Initialize a Linear Regression model
lin_reg = LinearRegression()

# Perform 5-fold cross-validation using KFold on the polynomial features
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(lin_reg, X_poly, Y, cv=kf, scoring='neg_mean_squared_error')
cv_rmse = np.sqrt(-np.mean(cv_scores))
print("Cross-validated RMSE (Polynomial degree 2):", cv_rmse)

# Get cross-validated predictions to assess potential overfitting
Y_pred_cv = cross_val_predict(lin_reg, X_poly, Y, cv=kf)
print("Cross-validated R2 (Polynomial degree 2):", r2_score(Y, Y_pred_cv))

# Train the final model on the full dataset using polynomial features
lin_reg.fit(X_poly, Y)
Y_pred = lin_reg.predict(X_poly)
print("Training RMSE (Polynomial degree 2):", np.sqrt(mean_squared_error(Y, Y_pred)))
print("Training R2 (Polynomial degree 2):", r2_score(Y, Y_pred))

# Print the regression coefficients and intercept
print("Regression coefficients:", lin_reg.coef_)
print("Regression intercept:", lin_reg.intercept_)

# Plot predictions vs. actual values for training and cross-validated predictions
plt.figure(figsize=(10, 6))
plt.plot(Y_LES, label='Actual eddy_double_LES', color='green')
plt.plot(Y, label='Actual eddy_double_TBNN', color='blue')
plt.plot(Y_pred, label='Poly Regression Prediction (Train)', color='red', linestyle='--')
plt.plot(Y_pred_cv, label='Poly Regression Prediction (CV)', color='orange', linestyle='-.')
plt.xlabel('Sample Index')
plt.ylabel('Value')
plt.title('Polynomial Regression (degree 2) Prediction vs. Actual')
plt.legend()
plt.grid(True)
plt.show()

# Interpolate the results onto the spatial grid
points = np.column_stack((x_point.ravel(), y_point.ravel()))
eddy_double_LES_interp = griddata(points, eddy_double_LES.ravel(), (x, y), method='cubic').reshape(x.shape)
eddy_double_TBNN_interp = griddata(points, eddy_double_TBNN.ravel(), (x, y), method='cubic').reshape(x.shape)
Y_pred_interp = griddata(points, Y_pred.ravel(), (x, y), method='cubic').reshape(x.shape)

# Set common color scale limits for the spatial plots
vmin = -0.05
vmax = 0.20

# Plot the interpolated maps in a vertical stack (3 subplots)
fig, axs = plt.subplots(3, 1, figsize=(8, 8))
im0 = axs[0].pcolormesh(x, y, eddy_double_LES_interp, shading='auto', cmap='jet', vmin=vmin, vmax=vmax)
axs[0].set_title('eddy double LES')
axs[0].set_aspect('equal', adjustable='box')
axs[0].set_xlim(np.min(x), np.max(x))
axs[0].set_ylim(np.min(y), np.max(y))

im1 = axs[1].pcolormesh(x, y, eddy_double_TBNN_interp, shading='auto', cmap='jet', vmin=vmin, vmax=vmax)
axs[1].set_title('eddy double TBNN')
axs[1].set_aspect('equal', adjustable='box')
axs[1].set_xlim(np.min(x), np.max(x))
axs[1].set_ylim(np.min(y), np.max(y))

im2 = axs[2].pcolormesh(x, y, Y_pred_interp, shading='auto', cmap='jet', vmin=vmin, vmax=vmax)
axs[2].set_title('Polynomial Regression Y pred')
axs[2].set_aspect('equal', adjustable='box')
axs[2].set_xlim(np.min(x), np.max(x))
axs[2].set_ylim(np.min(y), np.max(y))

fig.subplots_adjust(right=0.85, hspace=0.1)
cbar_ax = fig.add_axes([0.88, 0.15, 0.02, 0.7])
fig.colorbar(im2, cax=cbar_ax)
plt.show()

# Optionally, get a detailed regression summary using statsmodels
X_const = sm.add_constant(X_poly)
model = sm.OLS(Y, X_const).fit()
print(model.summary())

### edd viscosity = function(Sij)

In [ ]:
# Prepare features and target variables
# Use columns 0, 1, 3, and 4 from Sij as features
X = Sij[:, [0, 1, 3, 4]]
Y = eddy_double_TBNN.ravel()
Y_LES = eddy_double_LES.ravel()

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Use PolynomialFeatures to create higher order terms
# degree = 2 indicates quadratic terms (set degree to 2, 3, 4, etc. as needed)
poly = PolynomialFeatures(degree=4, include_bias=True)
X_poly = poly.fit_transform(X_scaled)

# Initialize a Linear Regression model
lin_reg = LinearRegression()

# Perform 5-fold cross-validation using KFold on the polynomial features
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(lin_reg, X_poly, Y, cv=kf, scoring='neg_mean_squared_error')
cv_rmse = np.sqrt(-np.mean(cv_scores))
print("Cross-validated RMSE (Polynomial degree 2):", cv_rmse)

# Get cross-validated predictions to assess potential overfitting
Y_pred_cv = cross_val_predict(lin_reg, X_poly, Y, cv=kf)
print("Cross-validated R2 (Polynomial degree 2):", r2_score(Y, Y_pred_cv))

# Train the final model on the full dataset using polynomial features
lin_reg.fit(X_poly, Y)
Y_pred = lin_reg.predict(X_poly)
print("Training RMSE (Polynomial degree 2):", np.sqrt(mean_squared_error(Y, Y_pred)))
print("Training R2 (Polynomial degree 2):", r2_score(Y, Y_pred))

# Print the regression coefficients and intercept
print("Regression coefficients:", lin_reg.coef_)
print("Regression intercept:", lin_reg.intercept_)

# Plot predictions vs. actual values for training and cross-validated predictions
plt.figure(figsize=(10, 6))
plt.plot(Y_LES, label='Actual eddy_double_LES', color='green')
plt.plot(Y, label='Actual eddy_double_TBNN', color='blue')
plt.plot(Y_pred, label='Poly Regression Prediction (Train)', color='red', linestyle='--')
plt.plot(Y_pred_cv, label='Poly Regression Prediction (CV)', color='orange', linestyle='-.')
plt.xlabel('Sample Index')
plt.ylabel('Value')
plt.title('Polynomial Regression (degree 2) Prediction vs. Actual')
plt.legend()
plt.grid(True)
plt.show()

# Interpolate the results onto the spatial grid
points = np.column_stack((x_point.ravel(), y_point.ravel()))
eddy_double_LES_interp = griddata(points, eddy_double_LES.ravel(), (x, y), method='cubic').reshape(x.shape)
eddy_double_TBNN_interp = griddata(points, eddy_double_TBNN.ravel(), (x, y), method='cubic').reshape(x.shape)
Y_pred_interp = griddata(points, Y_pred.ravel(), (x, y), method='cubic').reshape(x.shape)

# Set common color scale limits for the spatial plots
vmin = -0.05
vmax = 0.20

# Plot the interpolated maps in a vertical stack (3 subplots)
fig, axs = plt.subplots(3, 1, figsize=(8, 8))
im0 = axs[0].pcolormesh(x, y, eddy_double_LES_interp, shading='auto', cmap='jet', vmin=vmin, vmax=vmax)
axs[0].set_title('eddy double LES')
axs[0].set_aspect('equal', adjustable='box')
axs[0].set_xlim(np.min(x), np.max(x))
axs[0].set_ylim(np.min(y), np.max(y))

im1 = axs[1].pcolormesh(x, y, eddy_double_TBNN_interp, shading='auto', cmap='jet', vmin=vmin, vmax=vmax)
axs[1].set_title('eddy double TBNN')
axs[1].set_aspect('equal', adjustable='box')
axs[1].set_xlim(np.min(x), np.max(x))
axs[1].set_ylim(np.min(y), np.max(y))

im2 = axs[2].pcolormesh(x, y, Y_pred_interp, shading='auto', cmap='jet', vmin=vmin, vmax=vmax)
axs[2].set_title('Polynomial Regression Y pred')
axs[2].set_aspect('equal', adjustable='box')
axs[2].set_xlim(np.min(x), np.max(x))
axs[2].set_ylim(np.min(y), np.max(y))

fig.subplots_adjust(right=0.85, hspace=0.1)
cbar_ax = fig.add_axes([0.88, 0.15, 0.02, 0.7])
fig.colorbar(im2, cax=cbar_ax)
plt.show()

# Optionally, get a detailed regression summary using statsmodels
X_const = sm.add_constant(X_poly)
model = sm.OLS(Y, X_const).fit()
print(model.summary())

### edd viscosity = function(Sij, bij)

In [ ]:
# Prepare features and target variables
# Use columns 0, 1, 3, and 4 from Sij as features and bij_LES as additional features
X = np.concatenate((Sij[:, [0, 1, 3, 4]], bij_TBNN[:, [0, 1, 3, 4]]), axis=1)
Y = eddy_double_TBNN.ravel()
Y_LES = eddy_double_LES.ravel()

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Use PolynomialFeatures to create higher order terms
# degree = 2 indicates quadratic terms (set degree to 2, 3, 4, etc. as needed)
poly = PolynomialFeatures(degree=5, include_bias=True)
X_poly = poly.fit_transform(X_scaled)

# Initialize a Linear Regression model
lin_reg = LinearRegression()

# Perform 5-fold cross-validation using KFold on the polynomial features
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(lin_reg, X_poly, Y, cv=kf, scoring='neg_mean_squared_error')
cv_rmse = np.sqrt(-np.mean(cv_scores))
print("Cross-validated RMSE (Polynomial degree 2):", cv_rmse)

# Get cross-validated predictions to assess potential overfitting
Y_pred_cv = cross_val_predict(lin_reg, X_poly, Y, cv=kf)
print("Cross-validated R2 (Polynomial degree 2):", r2_score(Y, Y_pred_cv))

# Train the final model on the full dataset using polynomial features
lin_reg.fit(X_poly, Y)
Y_pred = lin_reg.predict(X_poly)
print("Training RMSE (Polynomial degree 2):", np.sqrt(mean_squared_error(Y, Y_pred)))
print("Training R2 (Polynomial degree 2):", r2_score(Y, Y_pred))

# Print the regression coefficients and intercept
print("Regression coefficients:", lin_reg.coef_)
print("Regression intercept:", lin_reg.intercept_)

# Plot predictions vs. actual values for training and cross-validated predictions
plt.figure(figsize=(10, 6))
plt.plot(Y_LES, label='Actual eddy_double_LES', color='green')
plt.plot(Y, label='Actual eddy_double_TBNN', color='blue')
plt.plot(Y_pred, label='Poly Regression Prediction (Train)', color='red', linestyle='--')
plt.plot(Y_pred_cv, label='Poly Regression Prediction (CV)', color='orange', linestyle='-.')
plt.xlabel('Sample Index')
plt.ylabel('Value')
plt.title('Polynomial Regression (degree 2) Prediction vs. Actual')
plt.legend()
plt.grid(True)
plt.show()

# Interpolate the results onto the spatial grid
points = np.column_stack((x_point.ravel(), y_point.ravel()))
eddy_double_LES_interp = griddata(points, eddy_double_LES.ravel(), (x, y), method='cubic').reshape(x.shape)
eddy_double_TBNN_interp = griddata(points, eddy_double_TBNN.ravel(), (x, y), method='cubic').reshape(x.shape)
Y_pred_interp = griddata(points, Y_pred.ravel(), (x, y), method='cubic').reshape(x.shape)

# Set common color scale limits for the spatial plots
vmin = -0.05
vmax = 0.20

# Plot the interpolated maps in a vertical stack (3 subplots)
fig, axs = plt.subplots(3, 1, figsize=(8, 8))
im0 = axs[0].pcolormesh(x, y, eddy_double_LES_interp, shading='auto', cmap='jet', vmin=vmin, vmax=vmax)
axs[0].set_title('eddy double LES')
axs[0].set_aspect('equal', adjustable='box')
axs[0].set_xlim(np.min(x), np.max(x))
axs[0].set_ylim(np.min(y), np.max(y))

im1 = axs[1].pcolormesh(x, y, eddy_double_TBNN_interp, shading='auto', cmap='jet', vmin=vmin, vmax=vmax)
axs[1].set_title('eddy double TBNN')
axs[1].set_aspect('equal', adjustable='box')
axs[1].set_xlim(np.min(x), np.max(x))
axs[1].set_ylim(np.min(y), np.max(y))

im2 = axs[2].pcolormesh(x, y, Y_pred_interp, shading='auto', cmap='jet', vmin=vmin, vmax=vmax)
axs[2].set_title('Polynomial Regression Y pred')
axs[2].set_aspect('equal', adjustable='box')
axs[2].set_xlim(np.min(x), np.max(x))
axs[2].set_ylim(np.min(y), np.max(y))

fig.subplots_adjust(right=0.85, hspace=0.1)
cbar_ax = fig.add_axes([0.88, 0.15, 0.02, 0.7])
fig.colorbar(im2, cax=cbar_ax)
plt.show()

# Optionally, get a detailed regression summary using statsmodels
X_const = sm.add_constant(X_poly)
model = sm.OLS(Y, X_const).fit()
print(model.summary())